# Modul 9 Persamaan Diferensial Numerik: PDP orde 1, persamaan transport/adveksi

Kembali ke [Persamaan Diferensial Numerik](./pdnum2024genap.qmd)

Ini adalah pertemuan terakhir praktikum Persamaan Diferensial Numerik tahun ini.